<a href="https://colab.research.google.com/github/renatoquiliche/DataScience-PUCRio-FB/blob/main/Test%202/Evaluation/Leonardo_Rafael.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#install.packages('mltools')
library(readr)
library(caret)
library(rpart)
library(rpart.plot)
library(mltools)
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [75]:
# Lendo os dados
salarios.c <- read.csv("https://raw.githubusercontent.com/renatoquiliche/DataScience-PUCRio-FB/main/Data/DS_salarios/ds_salaries.csv")


In [85]:
#### Pre-Processamento
### Exercicio 1 
# Transformando as variáveis categoricas em dummies exceto a variavel alvo "company_size"
str(salarios.c)
salarios.c$work_year <- as.character(salarios.c$work_year)
salarios.c$work_month <- as.character(salarios.c$work_month)
salarios.c$remote_ratio <- as.character(salarios.c$remote_ratio)
salarios.c$company_size <- factor(salarios.c$company_size)


dummies <- dummyVars(salary_in_usd ~ work_year + work_month + experience_level + employment_type + 
                       job_title + salary_currency + employee_residence + remote_ratio + 
                       company_location, data = salarios.c, fullRank = T)

# É utilizado fullRank = T para evitar existencia de variáveis com colinearidade perfeita no modelo
dummies <- predict(dummies,newdata=salarios.c)
salarios <- cbind(salarios.c,dummies)
salarios <- salarios[,-c(1,2,3,4,5,7,9,10,11)]
colnames(salarios) <- gsub(" ", "\\_", colnames(salarios))


# Criar amostras treino/teste 
set.seed(400)
inTrain <- createDataPartition(y=salarios$company_size,p=0.80,list=F)
train <- salarios[inTrain,]
test <- salarios[-inTrain,]

# Análise de variância, correlação e dependência linear
# NearZeroVar
nzv <- nearZeroVar(train, saveMetrics = F) 
train <- train[,-nzv]
# Removo 161 variaveis

'data.frame':	607 obs. of  13 variables:
 $ X                 : int  0 1 2 3 4 5 6 7 8 9 ...
 $ work_year         : chr  "2020" "2020" "2020" "2020" ...
 $ work_month        : chr  "6" "11" "12" "6" ...
 $ experience_level  : chr  "MI" "SE" "SE" "MI" ...
 $ employment_type   : chr  "FT" "FT" "FT" "FT" ...
 $ job_title         : chr  "Data Scientist" "Machine Learning Scientist" "Big Data Engineer" "Product Data Analyst" ...
 $ salary            : int  70000 260000 85000 20000 150000 72000 190000 11000000 135000 125000 ...
 $ salary_currency   : chr  "EUR" "USD" "GBP" "USD" ...
 $ salary_in_usd     : int  79833 260000 109024 20000 150000 72000 190000 35735 135000 125000 ...
 $ employee_residence: chr  "DE" "JP" "GB" "HN" ...
 $ remote_ratio      : chr  "0" "0" "50" "0" ...
 $ company_location  : chr  "DE" "JP" "GB" "HN" ...
 $ company_size      : Factor w/ 3 levels "L","M","S": 1 3 2 3 1 1 3 1 1 3 ...


In [87]:
# Correlacao
corr <- cor(train[c(1:2,4:33)],method = "spearman") 
alta_corr <- findCorrelation(corr, cutoff = .75, names=T) 
train <- dplyr::select(train,- alta_corr)
# Removo 5 variaveis

# Dependencia Linear
dep_lin <- findLinearCombos(train[,c(1:2,4:28)])   
dep_lin$remove
# Como dep_lin$remove é NULL, nao removemos nenhuma variavel

# Variaveis que foram mantidas
variaveis_mantidas <- colnames(train)



#### Processamento
### Exercicio 2 
## 2.1)
fitree <- rpart(company_size~., data=train, maxdepth = 3)

## 2.2)
rpart.plot(fitree)


### Exercicio 3 
## 3.1)
# A raiz da árvore identifica se o ano de trabalho é 2021 ou não. Se o ano for 2021, o nó subsequente identifica se o salário em dólares é maior ou igual a $19,000.00. Caso positivo, a folha é formada por 32% dos trabalhadores, sendo desses 59% de empresa L, 24% de M e 16% de empresa S. Se não for, a folha conclui que há somente 3% das pessoas nesta categoria, sendo 6% de empresa L, 44% de M e 50% de empresa S. Voltando ao primeiro nó, oriundo da raiz da árvore, caso o ano de trabalho seja 2020 ou 2022, o nó inclui a condição se ele trabalha parcialmente remoto ou não. Se sim, ele conclui que essa folha é formada por apenas 5% dos trabalhadores, sendo desses 62% de empresa L, 17% de empresa M e 21% de empresa S. Para o caso de não se trabalhar parcialmente remoto, a folha é formada por 59% das pessoas, sendo destes 17% de empresa L, 73% de M e 10% de empresa S. 


#### Pos-Processamento
### Exercicio 4 
## 4.1) Removendo as mesmas variaveis removidas na amostra treino
test <- dplyr::select(test, variaveis_mantidas)
# 4.1.1)
estimate <- predict(fitree, test, type = "class")
confusionMatrix(test$company_size, estimate)
# A partir da matriz de confusão, obteve-se uma acurácia de 71,67% para o modelo, um resultado satisfatório. Para empresas pequenas, pode-se concluir pela matriz que ela acertou a previsão em somente 1 dos 3 casos, tendo 2 como grandes. Para a previsão de empresas médias, a árvore operou bem melhor, prevendo 56 de 68 como médias, 4 como pequenas e 8 como grandes. Já para empresas grandes, de 49 empresas deste tamanho, ela categorizou 29 corretamente e 20 erroneamente, sendo 11 pequenas e 9 médias. 

# 4.1.2)
mcc(test$company_size,estimate)
# Foi obtido para o modelo um valor de MCC = 0,5. Este índice varia de -1 a 1, quanto mais próximo de 1, melhor o modelo, portanto pode-se considerar um bom valor.

ERROR: ignored